## Generate Query to Summary

### Generate Summary Query

In [1]:
import pandas as pd
import numpy as np

In [2]:
##Get all data from aws and make clusters to find similar curves
model_range = ["15","16","17","18","19","20","21","22"]

sql_claims = "\nunion".join("""
select veh.veh_line_cd, 20"""+mdl_yr+""" as mdl_yr, clm.prt_num_causl_base_cd, sum(clm.mtrl_cost)

    from dsc60117_aws_tz_db.clm_"""+str(mdl_yr)+""" clm, dsc60117_aws_tz_db.veh_"""+mdl_yr+""" veh
        where
            clm.vin_cd = veh.vin_cd
            and veh.sls_region_cd in ("SA", "NA")
            and clm.prt_num_causl_base_cd in ("7000","7902","6006","3504","9k546","6k682","6011","9U433","7A095","6K683","10655","19E616","5H270","18124")
    group by veh.veh_line_cd, clm.prt_num_causl_base_cd, 20"""+mdl_yr+"""
""" for mdl_yr in model_range)

with open("summary_clm.sql", "w") as f:
    sql_claims = """
set hive.execution.engine=tez;
set hive.optimize.index.filter=false;
set tez.task.resource.memory.mb=32000;
set hive.tez.container.size=32000;
set hive.tez.java.opts=-Xmx90g;
set tez.am.resource.memory.mb=32000;
set tez.task.resource.memory.mb=32000;
set tez.am.java.opts=-Xms60g;
"""+sql_claims
    f.write(sql_claims)

### Explore Summary

In [3]:
df_group = pd.read_csv("./data/summary_clm.tsv", sep="\t")
df_group_ = df_group.groupby(by=['veh_line_cd','prt_num_causl_base_cd']).agg('sum')
higher = df_group_.sort_values(by=['_c3'], ascending=False).reset_index()
#higher = higher[higher['_c3'] >= higher['_c3'].mean()]
higher = higher[higher['_c3'] > 0]

In [5]:
prt_num_list = list(higher.prt_num_causl_base_cd.unique())

veh_line_str = "(" + ",".join( '"'+v+'"' for v in higher.veh_line_cd.unique() ) + ")"
prt_num_cauls_base_str = "(" + ",".join( '"'+v+'"' for v in prt_num_list ) + ")"
#Do combination query
limit_query = ""

for i in range(len(higher.index)):
    row = higher.iloc[i]

    if row["prt_num_causl_base_cd"] == "*" or not row["prt_num_causl_base_cd"] in prt_num_list:
        continue
    
    if i == 0:
        limit_query = "(veh_line_cd = \""+row["veh_line_cd"]+"\" and prt_num_causl_base_cd = \""+row["prt_num_causl_base_cd"]+"\" )\n"
    else:
        limit_query += "or (veh_line_cd = \""+row["veh_line_cd"]+"\" and prt_num_causl_base_cd = \""+row["prt_num_causl_base_cd"]+"\" )\n"

### Claims:

In [9]:
##Get all data from aws and make clusters to find similar curves
model_range = ["15","16", "17", "18","19","20","21","22"]

sql_claims = "\nunion".join("""
    select clm.vin_cd, 20"""+(mdl_yr)+""" as mdl_yr, veh.veh_line_cd, clm.prt_num_causl_base_cd, clm.tis_wsd, clm.milge

        from dsc60117_aws_tz_db.clm_"""+(mdl_yr)+""" clm, dsc60117_aws_tz_db.prt_"""+(mdl_yr)+""" prt, dsc60117_aws_tz_db.veh_"""+(mdl_yr)+""" veh
            where
                prt.clm_key = clm.clm_key
                and clm.vin_cd = veh.vin_cd
                and veh.sls_region_cd in ("SA","NA")
                and veh.veh_line_cd in """+veh_line_str+"""
                and clm.prt_num_causl_base_cd in """+prt_num_cauls_base_str+"""
""" for mdl_yr in model_range) #model year: 2017-2019

sql_claims = """with db_1 as (
    """+sql_claims+"""
)
select * from db_1
    where
    """+limit_query+""""""

with open("sel_clm_SA_NA.sql", "w") as f:
    sql_claims = """
set hive.execution.engine=tez;
set hive.optimize.index.filter=false;
set tez.task.resource.memory.mb=32000;
set hive.tez.container.size=32000;
set hive.tez.java.opts=-Xmx90g;
set tez.am.resource.memory.mb=32000;
set tez.task.resource.memory.mb=32000;
set tez.am.java.opts=-Xms60g;
"""+sql_claims
    f.write(sql_claims)

### Production

In [6]:
sql_claims = "\nunion".join("""
select veh.vin_cd, 20"""+(mdl_yr)+""" as mdl_yr, veh.veh_line_cd, veh.eng_cd

    from dsc60117_aws_tz_db.veh_"""+(mdl_yr)+""" veh
        where
        veh.sls_region_cd in ("SA","NA")
        and veh.veh_line_cd in """+veh_line_str+"""
""" for mdl_yr in model_range) #model year: 2017-2019

with open("sel_prod_SA_NA.sql", "w") as f:
    sql_claims = """
set hive.execution.engine=tez;
set hive.optimize.index.filter=false;
set tez.task.resource.memory.mb=32000;
set hive.tez.container.size=32000;
set hive.tez.java.opts=-Xmx90g;
set tez.am.resource.memory.mb=32000;
set tez.task.resource.memory.mb=32000;
set tez.am.java.opts=-Xms60g;
"""+sql_claims
    f.write(sql_claims)